In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [12]:
x_train, y_train,x_test_original, y_test_original = ReadTS('CricketY')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
print(y_train)

(390, 300, 1)
[ 1.  3.  2.  2.  4.  0.  7.  9.  3.  4.  9.  1.  8.  4.  4.  9.  6. 10.
  6.  4.  1.  2.  2.  6.  0.  8.  5.  7. 11.  3. 10.  9.  8.  6.  4.  5.
 10.  7.  7.  9.  9. 11.  6.  2.  3.  6.  0. 11. 10. 11. 11.  8.  9.  3.
  1.  4.  1.  6.  4.  6. 11.  3.  2. 11.  7.  2.  9.  3.  6.  6.  0.  3.
 10.  7.  5.  2.  5.  0.  4.  8.  9. 11. 10.  4.  8.  4.  7. 10.  8. 11.
  1.  4. 10.  9.  3.  3.  6.  9.  9.  8.  0.  5.  4.  9.  7. 10. 11.  8.
  4. 11.  0.  0. 11. 11.  4.  2.  1.  6.  8.  6.  5.  2.  0. 10.  1.  1.
  9. 10.  7. 10.  4.  7.  7. 11.  1.  2.  3.  9.  2.  6.  8.  5.  6.  2.
  8.  8.  3.  2. 11.  3.  5.  9.  0.  5.  0.  0.  0.  3.  4.  4. 11. 10.
 11.  9.  0.  2.  6. 11.  4.  8.  1.  5.  9. 11.  5. 10.  0.  0.  5.  5.
  4.  9.  9.  1.  3.  4. 10.  9. 11.  2.  3.  2.  0.  3.  4.  6.  8.  3.
  7.  2.  6.  7.  8. 11.  7.  6.  0.  9.  8.  5.  4.  1.  3.  9.  6.  4.
  0.  1.  6.  5.  7. 10.  3.  1.  4. 10.  7.  8.  5. 10.  1. 10. 11.  7.
  2.  6.  1.  4.  4.  8.  2.  3.  5. 

In [4]:
model = make_CNN_model(x_train.shape[1:],num_classes = 12)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


2024-11-13 21:46:08.790274: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 1s 26ms/step - loss: 2.4401 - sparse_categorical_accuracy: 0.1250 - val_loss: 2.4809 - val_sparse_categorical_accuracy: 0.0641
Epoch 2/100
10/10 [==============================] - 0s 16ms/step - loss: 2.2736 - sparse_categorical_accuracy: 0.2372 - val_loss: 2.4680 - val_sparse_categorical_accuracy: 0.0769
Epoch 3/100
10/10 [==============================] - 0s 16ms/step - loss: 2.1904 - sparse_categorical_accuracy: 0.2788 - val_loss: 2.4584 - val_sparse_categorical_accuracy: 0.1538
Epoch 4/100
10/10 [==============================] - 0s 16ms/step - loss: 2.1316 - sparse_categorical_accuracy: 0.2917 - val_loss: 2.4550 - val_sparse_categorical_accuracy: 0.0897
Epoch 5/100
10/10 [==============================] - 0s 16ms/step - loss: 2.0755 - sparse_categorical_accuracy: 0.3301 - val_loss: 2.4612 - val_sparse_categorical_accuracy: 0.0897
Epoch 6/100
10/10 [==============================] - 0s 15ms/step - loss: 2.0288 - sparse_categorical_accuracy: 

In [5]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

13/13 [==============================] - 0s 4ms/step - loss: 2.2960 - sparse_categorical_accuracy: 0.3641
Test accuracy 0.3641025722026825
Test loss 2.2959907054901123


### Create perturbed test sets and calculate decrease in accuracy

In [6]:
print(y_test_original.shape)

(390,)


In [7]:
print(inspect.getsource(perturb))
print(inspect.signature(perturb))

def perturb(perturbation_strategy, ts, index0, index1, global_ts = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1)    
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    

(perturbation_strategy, ts, index0, index1, global_ts=[])


In [8]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])

x_test_rbp = np.asarray([perturb('RBP',x,200,300) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

pert1 = perturb('RBP1',perturb_test_set[0],200,300, global_ts = perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1',x,60,160, global_ts = perturb_test_set) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,200,300) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,200,300) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,200,300) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

13/13 [==============================] - 0s 3ms/step


In [16]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
l2norm(x_test_noise)
l2norm(x_test_blur)

4.724730047290539
11.258677143050921
6.046103068098732
20.7296495710107
0.6819394890242894


In [9]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [10]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [11]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.3641025641025641
 RBP Accuracy: 0.26666666666666666
 RBP1 Accuracy: 0.11794871794871795
 Zero Accuracy: 0.21794871794871795
 Noise Accuracy: 0.1
 Blur Accuracy: 0.30256410256410254



In [15]:
print(np.argmax(predictions_rbp,axis=1))

[ 3  3  3  3 10 10  3  3  3 10 10  3  4 10  3  4  3 10  3  3  3  3  3 10
  3 10 10 10 10  3  9  3  3  3  3  3  3  9  4  3  3  9 10 10  3  3 10  9
 10  9  1 10 10  9 10  3  4  9  3 10 10 10  9  3  3 10 10  1 10  4 10  3
  3  3  3  3  3 10  3  3  3  3  9  3  4 10  3 10  3  3  3 10 10  3  3 10
 10  3 10  3  3  9 10  4  3  3 10  3  3  3  3  9  3  3  3  3  3  9 10  3
  3  3 10  3  3  3  3 10  3  3  3 10 10  3 10  3  3  9 10  4 10  3  3  3
 10  3  3 10  3  9  3 10  9  3  3  3  9  3  3  3  1  9  3  3 10 10  3 10
 10  3  3 10  3 10 10  9  4  9  3  3 10  3  3  5  3  3  3  3  9 10  9  3
  1  9  3 10  3 10  9 10 10  3  3  3  3 10  3  9  9  3 10 10  3  3 10  4
  9  9 10 10  9  3  3  3 10  9  3  3  3 10  9  3 10 10 10 10 10  3  3  3
  3  9 10  3 10 10  3  3 10  3  3 10 10  4 10  3  3  3 10  9 10  3  3 10
  3 10  3 10 10 10 10  3  3  3  3 10  3 10 10 10  9 10  3  3  6  3  4 10
  3  9 10  3 10  3 10  9  3 10 10  3  3 10  9  5  4 10  3  3  3 10  3  9
 10  3  3  3 10  4  1  3  3  3  3  3  3  9 10 10  3